In [8]:
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from statsmodels.formula.api import ols

train = {}
test = {}
# Loading in all the data
for (curr_1, curr_2) in [('BTC', 'ETH'), ('BTC', 'LTC'), ('BTC', 'XRP'), ('ETH', 'LTC'), ('ETH', 'XRP'), ('LTC', 'XRP')]:
    for interval in ['1min', '10min', '60min', '1d']:
        train[curr_1 + '_' + curr_2 + '_' + interval] = pd.read_csv(curr_1 + '_' + interval + '_' + curr_2 + '_' + interval + '_march.csv')
        test[curr_1 + '_' + curr_2 + '_' + interval] = pd.read_csv(curr_1 + '_' + interval + '_' + curr_2 + '_' + interval + '_april.csv')

# Loading in all the models
models = {}
for (curr_1, curr_2) in [('BTC', 'ETH'), ('BTC', 'LTC'), ('BTC', 'XRP'), ('ETH', 'LTC'), ('ETH', 'XRP'), ('LTC', 'XRP')]:
    for interval in ['1min', '10min', '60min', '1d']:
        models[curr_1 + '_' + curr_2 + '_' + interval] = {}
        models[curr_1 + '_' + curr_2 + '_' + interval]['OLS'] = pickle.load(open('Models/' + curr_1 + '_' + curr_2 + '_' + interval + '_ols_1.pickle', 'rb'))
        models[curr_1 + '_' + curr_2 + '_' + interval]['SVR'] = pickle.load(open('Models/' + curr_1 + '_' + curr_2 + '_' + interval + '_svr.pickle', 'rb'))

In [19]:
# Implement a 1 - period predict then trade strategy
# After the following period, the trade is unwound and a new position is taken
# Important here is the notion of balanced risk, so we are just taking offsetting positions
# Assumptions: no transaction costs, ability to short-sell currencies (also without costs)
# Costs are platform and credit dependent, hence impossible to model

NOTIONAL = 1000
trades = {}

for key in models:
    curr_1 = key.split('_')[0]
    curr_2 = key.split('_')[1]
    trades[key] = {}
    for model in models[key]:
        print(f'SIMULATION: {key}')
        print(f'MODEL: {model}')
        trades[key][model] = {}
        if model == 'OLS':
            prediction = models[key][model].predict(test[key])
            test[key][f'{model} Predicted Price Ratio'] = prediction.values
        if model == 'SVR':
            # Can't save down the series transform fit
            scalar_x = StandardScaler()
            scalar_y = StandardScaler()
            scalar_x.fit(train[key][['Net Polarity', 'Net Subjectivity']])
            scalar_y.fit(train[key][['Price Ratio']])
            input_vec = scalar_x.transform(test[key][['Net Polarity', 'Net Subjectivity']])
            mask = ~np.isnan(input_vec) & ~np.isinf(input_vec)
            mask = [True if all(val) else False for val in mask]
            prediction = models[key][model].predict(input_vec[mask, :])
            prediction = scalar_y.inverse_transform(prediction)
            temp = np.empty((test[key][['Price Ratio']].size,1))
            temp[:] = np.nan
            test[key][f'{model} Predicted Price Ratio'] = temp.flatten()
            test[key][f'{model} Predicted Price Ratio'].loc[mask] = prediction
        print('Done predictions')
        # Now simulate the strategy using the predicted price and holding for one interval
        for period in test[key].index:
            # print('Simulating: ' + str(period))
            prediction_ratio = test[key][f'{model} Predicted Price Ratio'][period]
            actual_ratio = test[key]['Price Ratio'][period]
            if not np.isnan(test[key][f'{model} Predicted Price Ratio'][period]) and period != max(test[key].index):
                if actual_ratio < prediction_ratio:
                    curr_1_position = NOTIONAL / test[key][f'{curr_1} Price'][period]
                    curr_2_position = -1 * NOTIONAL / test[key][f'{curr_2} Price'][period]
                elif actual_ratio > prediction_ratio:
                    curr_1_position = -1 * NOTIONAL / test[key][f'{curr_1} Price'][period]
                    curr_2_position = NOTIONAL / test[key][f'{curr_2} Price'][period]
                else:
                    curr_1_position = 0
                    curr_2_position = 0

                trades[key][model][period] = {'Timestamp': test[key][f'Timestamp'][period], f'{curr_1} Traded': curr_1_position, f'{curr_2} Traded': curr_2_position, 'Profit': curr_1_position * test[key][f'{curr_1} Price'][period + 1] + curr_2_position * test[key][f'{curr_2} Price'][period + 1]}
                if period == 0:
                    trades[key][model][period]['Total Profit'] = trades[key][model][period]['Profit']
                else:
                    trades[key][model][period]['Total Profit'] = trades[key][model][period - 1]['Total Profit'] + trades[key][model][period]['Profit']
            else:
                trades[key][model][period] = {'Timestamp': test[key][f'Timestamp'][period], f'{curr_1} Traded': 0, f'{curr_2} Traded': 0, 'Profit': 0}
                if period == 0:
                    trades[key][model][period]['Total Profit'] = trades[key][model][period]['Profit']
                else:
                    trades[key][model][period]['Total Profit'] = trades[key][model][period - 1]['Total Profit'] + trades[key][model][period]['Profit']
        pd.DataFrame(trades[key][model]).transpose().to_csv(f'Results/{key}_{model}.csv', index=False)

SIMULATION: BTC_ETH_1min
MODEL: OLS
Done predictions
SIMULATION: BTC_ETH_1min
MODEL: SVR


C:\Users\dubcz\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


KeyboardInterrupt: 